**!!! ДАННЫЙ НОУТБУК ПРЕДНАЗНАЧЕН ТОЛЬКО ДЛЯ ЗАПУСКА В GOOGLE COLAB !!!**

# Детекция лиц в магазине (результаты)

# 1 Постановка задачи

Есть запись с видеокамеры внутри магазина. Требуется детектировать лица людей, которые находятся в пределах магазина. За пределами магазина детектор не должен срабатывать.

Написать скрипт, который на вход принимает файл с видео, выполняет детекцию и сохраняет видео с детекцией в отдельный файл.

Дополнительно необходимо считать уникальных людей, которые посетили магазин.

Ссылка на исходное видео:
https://drive.google.com/file/d/1tHsf2gosdpGGaBtBJRj7cPJAr3eTloD0

## 1.1 Возможные пути решения задачи

### 1.1.1 Определение расстояния от камеры до объекта с помощью карты глубины изображения (depth map)

Для данного пути требуется стереоизображение. Например, можно установить 2 одинаковых по техническим характеристикам веб-камеры с небольшим расстоянием друг от друга, и направить их в одну точку. В результате с помощью двух записей под разным углом можно вычислить карту глубины изображения. С помощью данной карты можно определить расстояние объекта до камеры.

В данной задаче запись ведется только с одной камеры, поэтому данным путем задачу решить не получится.

### 1.1.2 Определение расстояния от камеры до объекта с помощью размера бокса детекции лица на изображении

В данном случае предполагаем, что чем дальше человек от камеры, тем меньше будут размеры бокса детекции его лица. Можно считать площадь бокса. Если площадь меньше определенного порога, то такие боксы можно удалять. Такой подход можно применить к уже обученным детекторам.

В случае изменения расположения камеры будет достаточным поменять параметр, отвечающий за фильтрацию по площади. Так же для данного пути можно обучить детектор с нуля, взяв готовый большой датасет с разметкой лиц людей и добавить к нему изображения с камеры магазина с ручной разметкой. Так же важно учитывать мировую ситуацию с пандемией, люди ходят в масках. Требуется много примеров лиц с маской.

### 1.1.3 Определение человека в магазине и за его пределами с помощью обучения детектора с механизмом классификации

В этом случае можно обучить детектор находить лица людей и определять находится ли человек в магазине или за его пределами. Для этого требуется обучить детектор на изображениях с камеры магазина. И выполнить ручную разметку лиц 2 классами: IN (в магазине), OUT (вне магазина).

Данный подход требует большого количества примеров изображений для своей корректной работы. Также данный метод перестанет работать, если изменить ракурс камеры или переставить ее в другое место. Потребуется переобучение модели.

### 1.1.4 Ансамбль моделей

В данном подходе можно комбинировать разные модели между собой.
Например:
- предобученный детектор для детекции лиц;
- обученный с нуля детектор на основе подхода определения расстояния от камеры с помощью размера бокса детекции лица;
- обученный с нуля детектор на основе подхода человека в магазине и за его пределами с помощью обучения детектора с механизмом классификации;

Результаты работы моделей можно объединять в помощью механизма NMS или более прогрессивного Weighted Boxes Fusion.

# 2 Предложенное решение

В данном решении будет использована ансамблевая модель. Взяты следующие модели:

- предобученная модель детекции лиц MTCNN из фреймворка facenet (базовая модель)
- обученный с нуля детектор с механизмом классификации на основе архитектуры VarifocalNet из фреймворка MMDetection.

Ноутбук с обучением детектора можно найти здесь:
https://github.com/inject0r1945/ds_projects/blob/main/CV/shop_face_detection/train_vfnet_detector.ipynb

Модель MTCNN предобучена на большом датасете лиц и может работать как универсальный детектор. Но в видео для данной задачи не всегда определяет лица людей по следующим предполагаемым причинам:
- много людей в масках, модель не обучалась на таких лицах;
- качество изображения среднее, много темных тонов и недостаточно контрастности.

Для улучшения базовой модели принято решение обучить с нуля детектор архитектуры VarifocalNet и объединить его результаты с базовой моделью. 

Для этой задачи были отобраны кадры из видео для обучения и выполнена ручная разметка лиц с помощью сервиса makesense.ai. Выделялись лица как за пределами магазина, так и в магазине. Детектор был обучен на классификацию людей, как было описано в пункте 1.1.3. В обучении использовалось около 140 изображений.

Результаты базовой модели и дополнительной объединяются с помощью Weighted Boxes Fusion.

Базовая модель фильтруется по площади бокса. Дополнительная модель умеет определять лица в магазине, но для подстраховки задана минимальная фильтрация по площади. На новых данных дополнительная модель может не находить лица, так как для ее обучения требуется больше примеров с людьми в магазине.

Дополнительную модель можно обучать с помощью KFold-разбиения данных. В данной работе датасет был разбит на 3 фолда и в ансамбле дополнительная модель берется только для 1-го фолда для упрощения работы. 

Подсчет уникальных посетителей в данной работе не осущесвляется и предлагается только в виде следующего алгоритма действий:
- создаем базу данных и в отдельной таблице храним уникальные боксы лиц человека в виде эмбединга, которому автоматически присваивается целочисленный идентификатор;
- перед добавлением нового лица в таблицу существующих эмбедингов надо убедиться, что человек в ней отсутствует с помощью сравнения по метрики близости;
- в отдельной таблице храним связь даты посещения с идентификатором ембединга лица человека;
- дату посещения получаем из кадра видео во время детекции с помощью OCR;
- перед добавлением новой связи лица человека с датой, добавляем эмбединг лица в общую таблицу эмбедингов с указаными вышепроверками;
- для ведения базы берем детекции только внутри магазина, таким образом если сработала модель детекции, то считаем человеку посещение;
- чтобы засчитать человеку только одно посещение, требуется сравнить его ембединг лица со всему другими ембедингами лиц людей на дату детекции: если похожие люди не найдены, значит добавляем новую связь ембединга с датой, иначе ничего не пишем в базу.



# 3 Подготовка к работе

In [ ]:
!pip install facenet-pytorch
!pip install ensemble-boxes

## 3.1 Блок установки компонентов MMDetection

Выполняется до 40 минут.

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

# install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow==7.0.0

Looking in links: https://download.pytorch.org/whl/torch_stable.html
Cloning into 'mmdetection'...
remote: Enumerating objects: 23075, done.
remote: Total 23075 (delta 0), reused 0 (delta 0), pack-reused 23075
Receiving objects: 100% (23075/23075), 25.84 MiB | 32.08 MiB/s, done.
Resolving deltas: 100% (16157/16157), done.
/content/mmdetection
Obtaining file:///content/mmdetection
  Running setup.py develop for mmdet


In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.5.1+cu101 True
2.21.0
11.1
GCC 7.5


Загрузка весов обученной с нуля модели детектора с архитектурой VarifocalNet

In [ ]:
!mkdir checkpoints
!wget wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate \
           'https://docs.google.com/uc?export=download&id=1ZP9n79QCIIyA-oAIJRPp13XD1aCHXIJq' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1ZP9n79QCIIyA-oAIJRPp13XD1aCHXIJq" \
           -O checkpoints/vfnet_checkpoint.pth && rm -rf /tmp/cookies.txt

--2022-02-14 02:25:55--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2022-02-14 02:25:55--  https://docs.google.com/uc?export=download&confirm=OYQ1&id=1ZP9n79QCIIyA-oAIJRPp13XD1aCHXIJq
Resolving docs.google.com (docs.google.com)... 142.250.125.101, 142.250.125.138, 142.250.125.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.125.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-68-docs.googleusercontent.com/docs/securesc/bbjvap6m2ivaphvnavns9ncjh413idc9/glbchtl69jjqcuf14df8vqal9igdiupi/1644805500000/15088970906101947359/04791134259099198641Z/1ZP9n79QCIIyA-oAIJRPp13XD1aCHXIJq?e=download [following]
--2022-02-14 02:25:55--  https://doc-04-68-docs.googleusercontent.com/docs/securesc/bbjvap6m2ivaphvnavns9ncjh413idc9/glbchtl69jjqcuf14df8vqal9igdiupi/1644805500000/15088970906101947359/04791134259099198641Z/1ZP9n79QCIIyA-oAIJRPp13XD1aCHXIJq?e=d

## 3.2 Блок импорта необходимых библиотек и инициализация переменных

In [ ]:
import random
from time import sleep

import numpy as np
import pandas as pd

import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw
from tqdm import tqdm
from google.colab.patches import cv2_imshow
from facenet_pytorch import MTCNN, InceptionResnetV1
from mmdet.apis import inference_detector, init_detector
import mmcv
from mmcv import Config
from ensemble_boxes import weighted_boxes_fusion

SEED = 2021
random.seed(SEED)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Данные параметры менять нельзя:

In [ ]:
# Веса для модели детектора VarifocalNet
VFNET_MODEL_WEIGHTS = '/content/mmdetection/checkpoints/vfnet_checkpoint.pth'
# Папка с MMDetection, папка создается во время установки
MMDET_PATH = '/content/mmdetection'

## 3.3 Набор функций для выполнения обработки видео

In [ ]:
def rectangle_area(box):
    """Считаем площадь, ширину и высоту бокса
    
    Параметры
    ---------
    box : array
      Координаты одного бокса в виде массива в формате [x1, y1, x2, y2]

    Результат
    ---------
      area, box_width, box_height
    """
    box_width = box[2] - box[0]
    box_height = box[3] - box[1]
    area = box_width * box_height

    return area, box_width, box_height

def normalize_coordinates(x1, y1, x2, y2, img_width, img_height):
    """Нормализация координат бокса по ширине и высоте изображения
    
    Параметры
    ---------
    x1 : int
      Координата по оси x верхнего левого угла бокса
    y1 : int
      Координата по оси y верхнего левого угла бокса
    x2 : int
      Координата по оси x нижнего правого угла бокса
    y2 : int
      Координата по оси y нижнего правого угла бокса
    img_width : int
      Ширина изображения
    img_height : высота изображения

    Результат
    ---------
    x1, y1, x2, y2 : (float, float, float, float)
      Нормированные координаты бокса
    """
    x1, x2 = x1 / img_width,  x2 / img_width
    y1, y2 = y1 / img_height,  y2 / img_height
    return x1, y1, x2, y2

def denormalize_coordinates(x1, y1, x2, y2, img_width, img_height):
    """Денормализация координат бокса по ширине и высоте изображения
    
    Параметры
    ---------
    x1 : int
      Координата по оси x верхнего левого угла бокса
    y1 : int
      Координата по оси y верхнего левого угла бокса
    x2 : int
      Координата по оси x нижнего правого угла бокса
    y2 : int
      Координата по оси y нижнего правого угла бокса
    img_width : int
      Ширина изображения
    img_height : высота изображения

    Результат
    ---------
    x1, y1, x2, y2 : (float, float, float, float)
      Денормированные координаты бокса
    """
    x1, x2 = int(x1 * img_width),  int(x2 * img_width)
    y1, y2 = int(y1 * img_height),  int(y2 * img_height)
    return x1, y1, x2, y2

In [ ]:
def apply_brightness_contrast(img, brightness=0, contrast=0):
    """Добавляем яркости и контрастности для изображения
    
    Параметры
    ---------
    img : np.array
      Изображение
    brightness : float
      Уровень яркости, по умолчанию 0.
    contrast : float
      Уровень контрастности, по умолчанию 0.
    
    Результат
    ---------
    buf : np.array
     Изображение с изменением по яркости и контрастности
    """
    
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow)/255
        gamma_b = shadow
        
        buf = cv2.addWeighted(img, alpha_b, img, 0, gamma_b)
    else:
        buf = img.copy()
    
    if contrast != 0:
        f = 131*(contrast + 127)/(127*(131-contrast))
        alpha_c = f
        gamma_c = 127*(1-f)
        
        buf = cv2.addWeighted(buf, alpha_c, buf, 0, gamma_c)

    return buf

In [ ]:
def filter_box(boxes, probs, thresh=0.9, area_thr=0):
    """ Фильтрация боксов по площадки и вероятности достоверности класса

    Параметры
    ---------
    boxes : np.array
      Матрица боксов. Каждая строка матрицы - отдельный бокс.
      Строка имеет формат x1, y1, x2, y2.
    probs : np.array
      Массив вероятностей для каждого бокса размером 1xN, где N - количество
      боксов в boxes
    thresh : float
      Порог для фильтрации боксов по вероятности из prob. Принимает значения
      от 0 до 1. По умолчанию 0.9.
    area_thr : float
      Площадь бокса для фильтрации. Если площадь бокса будет меньше указанного
      значения, то бокс будет удален. По умолчанию 0.

    Результат
    ---------
    filtered_bboxes, filtered_probs, box_params
      filtered_bboxes - массив отфильтрованных боксов, если боксов нет, то
                        принимает значение None
      filtered_probs - массив отфильтрованных вероятностей боксов, если боксов нет, то
                       принимает значение None
      box_params - массив параметров боксов размером Nx3, где N - количество фильтрованных боксов.
                   У каждого бокса перечислены параметры площадки, ширины и высоты
    """

    assert len(boxes) == len(probs)
    assert 0.0<=thresh<=1.0

    filtered_bboxes = []
    filtered_probs = []
    box_params = []
    
    for box, prob in zip(boxes, probs):
        x1, y1, x2, y2 = box
        b_area, b_width, b_height = [round(float(val), 2) for val in rectangle_area(box)]
        # Условия исходят из того, что чем ближе к камере человек, тем больше его лицо и тем ниже располагается бокс
        if prob >= thresh and ((y2>=700 and area_thr<=b_area<=31000) or (y2<700 and area_thr<=b_area<13000)):
            filtered_bboxes.append(box.tolist())
            box_params.append([b_area, b_width, b_height])
            filtered_probs.append(prob)

    if len(filtered_bboxes) > 0:
        filtered_bboxes = np.array(filtered_bboxes)
        filtered_probs = np.array(filtered_probs)
        box_params = np.array(box_params)
    else:
        filtered_bboxes = None
        filtered_probs = None
        box_params = None

    return filtered_bboxes, filtered_probs, box_params

def draw_box(img, boxes, box_params, model_name=None):
    """ Рисуем боксы и дополнительную информацию на изображении
    
    Параметры
    ---------
    img : np.array
      Изображение
    boxes : np.array
      Матрица боксов. Каждая строка матрицы - отдельный бокс.
      Строка имеет формат x1, y1, x2, y2.
    box_params : np.array
    model_name : str
      Название модели, от которой получены боксы.
      По умолчанию None.

    Результат
    ---------
    img : np.array
    """

    img = img.copy()
    for box, box_param in zip(boxes, box_params):
        box = box.astype(int)
        b_area, b_width, b_height = box_param
        cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), (0,0,255), 6)
        cv2.putText(img, f"S: {b_area}", (int(box[0]), int(box[1])-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
        cv2.putText(img, f"W: {b_width}", (int(box[0]), int(box[1])-40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
        cv2.putText(img, f"H: {b_height}", (int(box[0]), int(box[1])-70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
        if model_name:
            cv2.putText(img, f"MODEL: {model_name}", (int(box[0]), int(box[1])-100), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
    return img

In [ ]:
def make_vfnet_config(mmdet_path=None):
    """Создаем конфиг для детектора VarifocalNet"""

    if mmdet_path is None:
        mmdet_path = '.'
    cfg = Config.fromfile(f'{mmdet_path}/configs/vfnet/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco.py')
    cfg.model.bbox_head.num_classes = 2 
    cfg.model.test_cfg.nms.iou_threshold = 0.05
    cfg.data.train.pipeline[2].img_scale = [(1333, 480), (1333, 960)]
    # Модель обучалась на изображениях 1333x960
    # Если разрешение теста увеличить до реального изображения из видео,
    # то будут плохо определяться классы (в магазине, вне магазина)
    # Менять не нужно, фреймворк автоматически подгонит размер бокса под реальный
    # размер изображения. А изображение будет сжато только для получения результата.
    cfg.data.val.pipeline[1].img_scale = (1333, 960)
    cfg.data.test.pipeline[1].img_scale = (1333, 960)
    cfg.gpu_ids = range(1)
    
    return cfg

def vfnet_result_split(vfnet_predict):
    """ Разделяем результат модели VarifocalNet
    на массив с боксами и массив с вероятностями

    Параметры
    ---------
    vfnet_predict : np.array
      Результат функции inference_detector

    Результат
    ---------
    boxes, probs : (np.array, np.array)
    """
    boxes = []
    probs = []
    # Берем только результаты для класса 1 (IN - в магазине)
    vfnet_predict = vfnet_predict[1]
    for x1, y1, x2, y2, conf in vfnet_predict:
        boxes.append([x1, y1, x2, y2])
        probs.append(conf)
    if len(boxes) == 0:
        boxes = None
        probs = None
    else:
        boxes = np.array(boxes)
        probs = np.array(probs)

    return boxes, probs

def weighted_boxes(mtcnn_boxes, mtcnn_probs, vfnet_boxes, vfnet_probs, image_width,
                   image_height, iou_thr=0.5):
    """ Объединяем боксы из моделей MTCNN и VarifocalNet с помощью 
    Weighted Box Fusion
    
    Параметры
    ---------
    mtcnn_boxes : np.array
      Матрица боксов изображения от модели MTCNN
    mtcnn_probs : np.array
      Массив вероятностей боксов изображения от модели MTCNN
    vfnet_boxes : np.array
      Матрица боксов изображения от модели VarofocalNet
    vfnet_probs : np.array
      Массив вероятностей боксов изображения от модели VarofocalNet
    image_width : int
      Ширина изображения
    image_height : int
      Высота изображения
    iou_thr : float
      Порог срабатывания по IoU для объединения боксов. По умолчанию 0.5.

    Результат
    ---------
    denorm_w_boxes : np.array
      Объединенные боксы
    """

    assert 0<=iou_thr<=1.0
    
    # Функция weighted_boxes_fusion требует нормализации координато боксов
    norm_mtcnn_boxes = [normalize_coordinates(x1, y1, x2, y2, image_width, image_height) \
                        for x1, y1, x2, y2 in mtcnn_boxes]
    norm_vfnet_boxes = [normalize_coordinates(x1, y1, x2, y2, image_width, image_height) \
                        for x1, y1, x2, y2 in vfnet_boxes]
    
    boxes_list = [norm_mtcnn_boxes, norm_vfnet_boxes]
    scores_list = [mtcnn_probs.tolist(), vfnet_probs.tolist()]
    labels_list = [[1]*len(mtcnn_boxes), [1]*len(vfnet_boxes)]

    w_boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, 
                                                    iou_thr=iou_thr, skip_box_thr=0.0001)

    denorm_w_boxes = [denormalize_coordinates(x1, y1, x2, y2, image_width, image_height) \
                      for x1, y1, x2, y2 in w_boxes]
    denorm_w_boxes = np.array(denorm_w_boxes)

    return denorm_w_boxes

# 4 Скрипт для сохранения детекции лиц в исходном видео

## 4.1 Входные параметры



In [ ]:
# Видео для анализа
VIDEO_PATH = '/content/drive/MyDrive/data_science/FrontDoor.mp4'
# В какой файл будет сохранено видео с разметкой (у файла должно быть расширение mp4)
TARGET_VIDEO_PATH = '/content/drive/MyDrive/data_science/FrontDoor_with_detection.mp4'

# Если True, то будет использован ансамбль MTCNN + VarifocalNet,
# иначе только детекции модели MTCNN
USE_ENSEMBLE = True

## 4.2 Инициализация моделей

In [ ]:
# Иницализация детектора архитектуры VarifocalNet
vfnet_config = make_vfnet_config(mmdet_path=MMDET_PATH)
vfnet_model = init_detector(vfnet_config, VFNET_MODEL_WEIGHTS, device='cuda:0')

# Инициализация детектора архитектуры MTCNN
thresholds = [0.7, 0.8, 0.8]
mtcnn = MTCNN(keep_all=True, 
              select_largest=False, 
              thresholds=thresholds, 
              device='cuda:0',
              min_face_size=60)

load checkpoint from local path: /content/mmdetection/checkpoints/vfnet_checkpoint.pth


2022-02-14 02:26:19,814 - root - INFO - ModulatedDeformConvPack backbone.layer2.0.conv2 is upgraded to version 2.
2022-02-14 02:26:19,820 - root - INFO - ModulatedDeformConvPack backbone.layer2.1.conv2 is upgraded to version 2.
2022-02-14 02:26:19,825 - root - INFO - ModulatedDeformConvPack backbone.layer2.2.conv2 is upgraded to version 2.
2022-02-14 02:26:19,829 - root - INFO - ModulatedDeformConvPack backbone.layer2.3.conv2 is upgraded to version 2.
2022-02-14 02:26:19,834 - root - INFO - ModulatedDeformConvPack backbone.layer3.0.conv2 is upgraded to version 2.
2022-02-14 02:26:19,840 - root - INFO - ModulatedDeformConvPack backbone.layer3.1.conv2 is upgraded to version 2.
2022-02-14 02:26:19,845 - root - INFO - ModulatedDeformConvPack backbone.layer3.2.conv2 is upgraded to version 2.
2022-02-14 02:26:19,850 - root - INFO - ModulatedDeformConvPack backbone.layer3.3.conv2 is upgraded to version 2.
2022-02-14 02:26:19,855 - root - INFO - ModulatedDeformConvPack backbone.layer3.4.conv2 

## 4.3 Запуск обработки и сохранения видео

К рамке боксов прилагается отладочная информация:
- площадь бокса;
- высота и ширина бокса;
- какой модели бокс принадлежит.

In [ ]:
# Инициализируем захват видео
v_cap = cv2.VideoCapture(VIDEO_PATH)
# Определяем ширину и высоту кадра
frame_width = int(v_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(v_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# Определяем FPS источника видео
fps = int(v_cap.get(cv2.CAP_PROP_FPS))
# Инициализируем объект для записи видео
fourcc = cv2.VideoWriter_fourcc(*'FMP4')
v_writer = cv2.VideoWriter(TARGET_VIDEO_PATH, fourcc, fps, 
                           (frame_width, frame_height), True)

assert isinstance(frame_width, int)
assert isinstance(frame_height, int)
assert isinstance(fps, int)

try:
    # Цикл по каждому кадру видео
    for idx in tqdm(range(int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT)))):

        # Считываем текущий кадр
        success, frame = v_cap.read()
        # Прерываем цикл в случае ошибки чтения кадра
        if not success:
            break

        mtcnn_img = Image.fromarray(cv2.cvtColor(frame.copy(), cv2.COLOR_BGR2RGB))
        if USE_ENSEMBLE:
            vfnet_img = mmcv.imread(frame)

        # Получаем результат детекции модели MTCNN
        mtcnn_boxes, mtcnn_probs = mtcnn.detect(mtcnn_img)
        # Получаем результат детекции модели VarifocalNet
        if USE_ENSEMBLE:
            vfnet_boxes, vfnet_probs = vfnet_result_split(inference_detector(vfnet_model, vfnet_img))

        # Если найдены боксы в модели mtcnn, то выполняем их фильтрацию
        if mtcnn_boxes is not None:
            mtcnn_boxes, mtcnn_probs, mtcnn_box_params = filter_box(mtcnn_boxes, mtcnn_probs, thresh=0.9, area_thr=8000)
        # Если найдены боксы в модели vfnet, то выполняем их фильтрацию
        if vfnet_boxes is not None and USE_ENSEMBLE:
            vfnet_boxes, vfnet_probs, vfnet_box_params = filter_box(vfnet_boxes, vfnet_probs, thresh=0.7, area_thr=1000)

        # Здесь происходит объединение результатов двух моделей
        if USE_ENSEMBLE and vfnet_boxes is not None and mtcnn_boxes is not None:
            result_boxes = weighted_boxes(mtcnn_boxes, mtcnn_probs, vfnet_boxes, vfnet_probs, 
                                        iou_thr=0.05, image_width=frame_width, 
                                        image_height=frame_height)
            result_box_params = np.array([rectangle_area(box) for box in result_boxes])
            model_name = 'ENSEMBLE'
        elif USE_ENSEMBLE and vfnet_boxes is not None:
            result_boxes = vfnet_boxes
            result_box_params = vfnet_box_params
            model_name = 'VFNET'
        elif mtcnn_boxes is not None:
            result_boxes = mtcnn_boxes
            result_box_params = mtcnn_box_params
            model_name = 'MTCNN'
        else:
            result_boxes = None
            
        # Применяем результаты детекции к кадру
        if result_boxes is not None:
            frame = draw_box(frame, result_boxes, result_box_params, model_name=model_name)
                
        v_writer.write(frame)

finally:
    v_writer.release()

  5%|▌         | 2461/45045 [21:02<6:07:00,  1.93it/s]/usr/local/lib/python3.7/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:73: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
 21%|██        | 9567/45045 [1:21:33<5:06:07,  1.93it/s]/usr/local/lib/python3.7/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:85: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
 68%|██████▊   | 30562/45045 [4:19:25<2:05:36,  1.92it/s]

# 5 Какие подходы не сработали
- Связка основной модели и трекера, после первой детекции лица начинаем его плавный трекинг (у основной модели есть ложные срабатывания, есть проблема с удалением рамки после пропадания лица из кадра);
- Увеличение яркости и контракстности для основной модели (качество не изменилось);
- Увеличение разрешения изображения для основной модели (качество не изменилось);
- Предсказания дополнительной модели на разрешении в 2 раза большем, чем при обучении (ухудшилось качество классификации)

# 6 Что можно улучшить

- обучить дополнительную модель на большем количестве примеров лиц в магазине;
- улучшить качество разметки для дополнительной модели (более точные грани лица с минимальным захватом посторонних элементов на улице и в магазине);
- установить в ручной разметке класс IN (в магазине) для дополнительной модели после того, как человек перешел порог (сейчас класс IN определяется для человека на пороге, это место неоднозначное, есть ложные срабатыванния за порогом);
- обучить дополнительную модель на большом размеченном датасете с лицами (желательно, чтобы были лица в масках, добавить к этому датасету изображения ручной разметки из видео);
- попробовать детектировать не лицо, а фигуру человека (например, с помощью Detectron2);
- для ускорения вычислений можно взять в качестве дополнительной более легковесную модель (например, Yolov3);
- поставить в магазин две камеры для возможности генерации карты глубины.